영상이나 image 파일을 읽고 pose.process로 나온 result를 확인하면 result내부에 landmarks 정보가 저장되어 있다.
result안에 저장되어 있는 landmarks의 순서는 1~33번까지 BlazePose 표에 나와있는 순서대로 저장되어 있고, 33번까지 순서대로 가져와서 저장해주면 왼쪽 눈부터 시작해서 발 끝까지의 landmarks x,y,z,visibillity 정보를 확인할 수 있다.

In [1]:
import cv2
import mediapipe as mp
import pandas as pd

# MediaPipe에서 포즈 관련 유틸리티와 스타일을 불러옵니다.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# 포즈 데이터를 저장할 DataFrame을 생성합니다.
df = pd.DataFrame()

# 'video/sample.mp4'라는 비디오 파일을 읽기 위해 VideoCapture 객체를 생성합니다.
cap = cv2.VideoCapture('video/sample.mp4')

# MediaPipe 포즈 객체를 초기화합니다.
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  # 비디오가 열려있는 동안 계속 반복합니다.
  while cap.isOpened():
    # 비디오에서 프레임을 읽어옵니다.
    success, image = cap.read()
    # 프레임이 성공적으로 읽히지 않은 경우, 반복을 종료합니다.
    if not success:
      print("비디오 처리가 완료되었습니다.")
      break

    # 성능 향상을 위해 이미지를 수정 불가능하게 설정합니다.
    image.flags.writeable = False
    # 이미지를 BGR에서 RGB로 변환합니다.
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # 포즈 감지를 수행합니다.
    results = pose.process(image)

    # 포즈 랜드마크 데이터를 저장할 리스트를 초기화합니다.
    x = []
    # 감지된 각 랜드마크에 대한 정보를 추출하여 리스트에 추가합니다.
    for k in range(33):
        x.append(results.pose_landmarks.landmark[k].x)
        x.append(results.pose_landmarks.landmark[k].y)
        x.append(results.pose_landmarks.landmark[k].z)
        x.append(results.pose_landmarks.landmark[k].visibility)

    # 리스트를 DataFrame으로 변환하고 기존 DataFrame에 추가합니다.
    tmp = pd.DataFrame(x).T
    df = pd.concat([df, tmp])

    # 이미지를 수정 가능하게 하고 다시 BGR로 변환합니다.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # 포즈 랜드마크를 이미지 위에 그립니다.
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # 이미지를 화면에 표시합니다.
    cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
    # ESC 키를 누르면 반복을 종료합니다.
    if cv2.waitKey(5) & 0xFF == 27:
      break

# VideoCapture 객체를 해제합니다.
cap.release()

# 비디오 처리가 완료된 후 DataFrame에 저장된 데이터를 화면에 출력하고 'pose_data.csv' 파일로 저장합니다.
print(df)
df.to_csv('pose_data.csv')


Video processing complete.
         0         1         2         3         4         5         6    \
0   0.501562  0.265780 -0.591723  0.999703  0.516638  0.234704 -0.557032   
0   0.501637  0.265216 -0.584116  0.999701  0.516983  0.234685 -0.546942   
0   0.501724  0.264970 -0.601505  0.999705  0.517286  0.234676 -0.564227   
0   0.501790  0.264884 -0.611735  0.999708  0.517522  0.234661 -0.575460   
0   0.501923  0.264677 -0.612140  0.999711  0.517770  0.234540 -0.575999   
..       ...       ...       ...       ...       ...       ...       ...   
0   0.502255  0.268498 -0.566182  0.999640  0.519554  0.235198 -0.527793   
0   0.502271  0.268408 -0.563842  0.999629  0.519556  0.235150 -0.525541   
0   0.502279  0.268324 -0.562648  0.999619  0.519543  0.235090 -0.524321   
0   0.502290  0.268258 -0.562528  0.999616  0.519534  0.235018 -0.524168   
0   0.502308  0.268113 -0.561925  0.999610  0.519531  0.234818 -0.523568   

         7         8         9    ...       122       123   